In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:
# we use this function to convert responses to dataframes
def df_from_response(res):
    # initialize temp dataframe for batch of data in response
    df = pd.DataFrame()

    # loop through each post pulled from res and append to df
    for post in res.json()['data']['children']:
        df = df.append({
            'subreddit': post['data']['subreddit'],
            'title': post['data']['title'],
            'selftext': post['data']['selftext'],
            'upvote_ratio': post['data']['upvote_ratio'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'score': post['data']['score'],
            'link_flair_css_class': post['data']['link_flair_css_class'],
            'created_utc': datetime.fromtimestamp(post['data']['created_utc']).strftime('%Y-%m-%dT%H:%M:%SZ'),
            'id': post['data']['id'],
            'kind': post['kind']
        }, ignore_index=True)

    return df


In [4]:
# authenticate API
client_auth = requests.auth.HTTPBasicAuth('C0hV0oPDS4zxHVqzEhcUVw', '8Z_rDlP554yqq8UYKjH2SVW6JOoaYg')
data = {
    'grant_type': 'password',
    'username': 'paysthebills',
    'password': 'f89eoufX!@#'
}
headers = {'User-Agent': 'text as data/0.0.1'}

# send authentication request for OAuth token
res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=data, headers=headers)

# extract token from response and format correctly
TOKEN = f"bearer {res.json()['access_token']}"
# update API headers with authorization (bearer token)
headers = {**headers, **{'Authorization': TOKEN}}

In [14]:
# initialize dataframe and parameters for pulling data in loop
data = pd.DataFrame()
params = {'limit': 100}


# loop through 10 times (returning 1K posts)
for i in range(10):
    # make request
    res = requests.get("https://oauth.reddit.com/r/Bitcoin/hot",
                       headers=headers,
                       params=params)

    # get dataframe from response
    new_df = df_from_response(res)
    
    if new_df.shape[0] > 0:
        
        # take the final row (oldest entry)
        row = new_df.iloc[len(new_df)-1]
        # create fullname
        fullname = row['kind'] + '_' + row['id']
        # add/update fullname in params
        params['after'] = fullname
        
        # append new_df to data
        data = data.append(new_df, ignore_index=True)

In [15]:
data.shape

(784, 11)